In [1]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import torch.nn.functional as F
from tqdm import tqdm
from IPython.display import Audio
from scipy.io import wavfile
import soundfile
import librosa
import torch
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Model,Wav2Vec2Config, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

In [3]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger

In [4]:
# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

In [5]:
# from torchmetrics import F1
# from torchmetrics import Accuracy

from sklearn.metrics import f1_score

In [6]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--train_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/',     help='train feature path')
parser.add_argument('--valid_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/',     help='validation feature path')
parser.add_argument('--output_folder',    default='./output_folder_LCNN',     help='output folder')
parser.add_argument("--mode",default="murmur", type=str, help="mode : murmur/outcome")
parser.add_argument("--lr",default=1e-6, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=6 ,type=int)
parser.add_argument("--minibatchsize_valid", default=4, type=int)
parser.add_argument("--train_num_workers", default=8, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_LCNN', type=str)
parser.add_argument("--logdir", default='./log_hmd_LCNN/', type=str)
parser.add_argument("--model_name", default='hmd_LCNN_model_murmur', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=80, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [82]:
def feature_extract_raw(fnm, samp_sec=30, sr = 4000, pre_emphasis = 0):

    if isinstance(fnm, str) :  #fnm이 str타입이면 
        data, sample_rate = librosa.load(fnm, sr = sr)  # raw data와 sr 로딩
        data = data * 1.0    # 아마 float으로 바꾸기 위함
        data = data[sr:-sr]
    else :
        data = fnm * 1.0   # fnm이 raw data 일때.
        data = data[sr:-sr]
        sample_rate = sr    # 4000인 sr을 가져옴

    if samp_sec:     
        if len(data) > sample_rate * samp_sec :    # data가 20초 넘으면
            n_samp = len(data) // int(sample_rate * samp_sec)   # 얼마나 넘는지 보고 (데이터에 20초가 몇번 들어가나)
            signal = []
            for i in range(n_samp) :  # n번만큼 signal list 원소로 20초씩 자른 데이터를 넣음
                signal.append(data[ int(sample_rate * samp_sec)*i:(int(sample_rate * samp_sec)*(i+1))])
        else :
            n_samp = 1
            signal = np.zeros(int(sample_rate*samp_sec,))  # 빈 20초 signal 생성
            for i in range(int(sample_rate * samp_sec) // len(data)) : # 20초 안에 data가 몇번 들어가는지 만큼 루프
                signal[(i)*len(data):(i+1)*len(data)] = data      # data를 signal에 반복해서 넣어줌
            num_last = int(sample_rate * samp_sec) - len(data)*(i+1)  # n번 넣고 남은 나머지 공백을 계산
            signal[(i+1)*len(data):int(sample_rate * samp_sec)] = data[:num_last]  # 나머지 공백을 data를 잘라서 넣어줌
            signal = signal
    else:
        n_samp = 1
        signal = [data]

    for i in range(n_samp) :   # 위의 n_samp을 가져와서 루프
        if pre_emphasis :      # pre-emphasis 진행 보통 y(t) = x(t) - a*x(t-1)  근데 0이라서 pre-emphasis 없는듯
            emphasized_signal = np.append(signal[i][0], signal[i][1:] - pre_emphasis * signal[i][:-1])
        else :  # pre-emphasis 없음
            emphasized_signal = signal[i]

    return signal #n_samp

In [7]:
def read_recording(wav_fn, audio_length= 20, sr= 4000):  #20초 

    recording, frequency = load_wav_file(wav_fn)
    recording = recording * 1.0
    recording = recording[4000:-4000]

    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording

In [16]:
np.pad(np.array([1,2,3]), (0, 5), 'wrap')

array([1, 2, 3, 1, 2, 3, 1, 2])

In [86]:
wav_files = glob.glob(args.train_data_folder + '*.wav')

In [90]:
data, sample_rate = librosa.load(wav_files[0], sr = 4000)

In [100]:
read_recording(wav_files[0]).shape

(120000,)

In [94]:
data[4000:-4000].shape

(57536,)

In [3]:
torch.cuda.empty_cache()

In [8]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class Wav2Vec2_LCNN(nn.Module):

    def __init__(self,mode):

        super(Wav2Vec2_LCNN, self).__init__()
        '''input : (B,T)'''
        
        self.mode = mode
        self.wav2vec2_CTC = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
        self.wav2vec2_CTC.lm_head = nn.Identity()

        
        
        self.conv2d_1  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.conv2d_2  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        
        self.max2d_1 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_3  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_4  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_1 = nn.BatchNorm2d(32)
        
        self.conv2d_5  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_6  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_2 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        self.bn2d_2 = nn.BatchNorm2d(48)
        
        
        self.conv2d_7  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        self.conv2d_8  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_3 = nn.BatchNorm2d(48)
        
        self.conv2d_9  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_10  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_3 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_11  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        self.conv2d_12  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_4 = nn.BatchNorm2d(64)
        
        self.conv2d_13  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_14  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.bn2d_5 = nn.BatchNorm2d(32)
        
        
        self.conv2d_15  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_16  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_6 = nn.BatchNorm2d(32)
        
        
        self.conv2d_17  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_18  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.max2d_4 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.adaptavg = nn.AdaptiveAvgPool2d(1)
        
        
#        self.fc1 = nn.Linear(64, 128)
        
        if self.mode == 'murmur' :
            self.fc_mur = nn.Sequential(
                                        nn.Linear(32,1),
                                        nn.Sigmoid()
                                        )
        elif self.mode == 'outcome' :
            self.fc_out = nn.Sequential(
                                        nn.Linear(128,32),
                                        nn.ReLU(),
                                        nn.Linear(32,1),
                                        nn.Sigmoid()
                                        )
        
        
    def forward(self, input):
        
        input = self.wav2vec2_CTC(input).logits
        input = input.unsqueeze(1)
        
        x1 = self.conv2d_1(input)
        x2 = self.conv2d_2(input)
        max1 = torch.maximum(x1,x2)
        x = self.max2d_1(max1)
        
        x3 = self.conv2d_3(x)
        x4 = self.conv2d_4(x)
        max2 = torch.maximum(x3,x4)
        x = self.bn2d_1(max2)
        
        x5 = self.conv2d_5(x)
        x6 = self.conv2d_6(x)
        max3 = torch.maximum(x5,x6)
        x = self.max2d_2(max3)
        x = self.bn2d_2(x)
        
        x7 = self.conv2d_7(x)
        x8 = self.conv2d_8(x)
        max4 = torch.maximum(x7,x8)
        x = self.bn2d_3(max4)
        
        x9 = self.conv2d_9(x)
        x10 = self.conv2d_10(x)
        max5 = torch.maximum(x9,x10)
        x = self.max2d_3(max5)
        
        x11 = self.conv2d_11(x)
        x12 = self.conv2d_12(x)
        max6 = torch.maximum(x11,x12)
        x = self.bn2d_4(max6)
        
        x13 = self.conv2d_13(x)
        x14 = self.conv2d_14(x)
        max7 = torch.maximum(x13,x14)
        x = self.bn2d_5(max7)
        
        x15 = self.conv2d_15(x)
        x16 = self.conv2d_16(x)
        max8 = torch.maximum(x15,x16)
        x = self.bn2d_6(max8)
        
        x17 = self.conv2d_17(x)
        x18 = self.conv2d_18(x)
        max9 = torch.maximum(x17,x18)
        x = self.max2d_4(max9)
        
        x = self.adaptavg(x)
        x = x.view(x.size(0),-1)
        
#        x = self.fc1(x)
        
        
        if self.mode == 'murmur' :
            out = self.fc_mur(x)
        elif self.mode == 'outcome' :
            out = self.fc_out(x)
#        out = {'murmur':out1, 'outcome':out2}
        
        return out

In [11]:
model = Wav2Vec2_LCNN("murmur")
x = torch.randn((32,120000))
output = model(x)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
output

tensor([[0.6074],
        [0.6141],
        [0.6126],
        [0.6075],
        [0.6128],
        [0.6112],
        [0.6145],
        [0.6069],
        [0.6158],
        [0.6094],
        [0.6154],
        [0.6117],
        [0.6135],
        [0.6177],
        [0.6133],
        [0.6098],
        [0.6097],
        [0.6113],
        [0.6088],
        [0.6108],
        [0.6251],
        [0.6153],
        [0.6092],
        [0.6103],
        [0.6137],
        [0.6114],
        [0.6109],
        [0.6161],
        [0.6063],
        [0.6133],
        [0.6115],
        [0.6069]], grad_fn=<SigmoidBackward0>)

In [25]:
patient_file = glob.glob(args.train_data_folder + '*.txt')

In [26]:
patient_file

['/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/50103.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/50077.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/68298.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/51331.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/69152.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/85239.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/84949.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/85159.txt',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/31737.txt',
 '/home/jh20/Data/nr_data/ECG/Physion

In [27]:
patient_data = load_patient_data(patient_file[0])

In [30]:
0 == 0.0patient_data

'50103 4 4000\nAV 50103_AV.hea 50103_AV.wav 50103_AV.tsv\nPV 50103_PV.hea 50103_PV.wav 50103_PV.tsv\nTV 50103_TV.hea 50103_TV.wav 50103_TV.tsv\nMV 50103_MV.hea 50103_MV.wav 50103_MV.tsv\n#Age: Child\n#Sex: Male\n#Height: 128.0\n#Weight: 25.1\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Normal\n#Campaign: CC2014\n#Additional ID: nan\n'

In [61]:
feature_lst=[]
current_patient_data = load_patient_data(patient_file[0])
num_locations = get_num_locations(current_patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]
for j in range(num_locations):
    entries = recording_information[j].split(' ')
    recording_file = entries[2]
    filename = os.path.join(args.train_data_folder, recording_file)
    raw_data = feature_extract_raw(filename)  # (120000,)
    raw_data = raw_data.reshape((1,120000))
    feature_lst.append(raw_data)

In [72]:
filename

'/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/50103_MV.wav'

In [74]:
raw_data = raw_data.reshape((1,raw_len)).reshape((1,120000))

In [75]:
raw_data

array([[ 0.00280762,  0.00497437,  0.00360107, ..., -0.00515747,
        -0.00619507, -0.009552  ]])

In [76]:
recording_information = patient_data.split('\n')[1:4+1]

In [32]:
recording_information

['AV 50103_AV.hea 50103_AV.wav 50103_AV.tsv',
 'PV 50103_PV.hea 50103_PV.wav 50103_PV.tsv',
 'TV 50103_TV.hea 50103_TV.wav 50103_TV.tsv',
 'MV 50103_MV.hea 50103_MV.wav 50103_MV.tsv']

In [9]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math
import pickle


def read_recording(wav_fn, audio_length= 20, sr= 4000):  #20초 

    recording, frequency = librosa.load(wav_fn)
    recording = recording * 1.0
    recording = recording[4000:-4000]

    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording




class train_loader(Dataset):
    def __init__(self,data_folder, **kwargs):
        self.data_folder = data_folder
#         self.feature_folder = feature_folder
        
        self.patient_file = glob.glob(data_folder + '*.txt')
#         self.feature_file = glob.glob(feature_folder + '*.pickle')
        self.wav_files = glob.glob(data_folder + '*.wav')
#         self.num_feature_files = len(self.feature_file)
        
        self.murmur_classes = ['Present', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
        
        murmur_lable = []
        outcome_lable = []
        feature_lst = []
        for i in range(len(self.patient_file)):
            current_patient_data = load_patient_data(self.patient_file[i])
            num_locations = get_num_locations(current_patient_data)
            recording_information = current_patient_data.split('\n')[1:num_locations+1]
            for j in range(num_locations):
                entries = recording_information[j].split(' ')
                recording_file = entries[2]
                filename = os.path.join(data_folder, recording_file)
                raw_data = read_recording(filename)  # (120000,)
#                 raw_len = len(raw_data)
#                 raw_data = raw_data.reshape((1,raw_len))
                feature_lst.append(raw_data)
                
                if get_murmur(current_patient_data) == self.murmur_classes[0]:
                    murmur_lable.append(1)
                elif get_murmur(current_patient_data) == self.murmur_classes[1]:
                    murmur_lable.append(0)
                else :
                    murmur_lable.append(0)
                
                if get_outcome(current_patient_data) == self.outcome_classes[0]:
                    outcome_lable.append(1)
                elif get_outcome(current_patient_data) == self.outcome_classes[1]:
                    outcome_lable.append(0)
                
                

        
#         murmur_lable = np.zeros(self.num_feature_files, dtype=np.float32)
#         outcome_lable = np.zeros(self.num_feature_files, dtype=np.float32)
#         examples = {}
#         for i in range(self.num_feature_files):
#             filename = self.feature_file[i]
#             current_patient_id = filename.split('/')[-1].split('_')[0]
#             current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
#             with open(current_txt_file, 'r') as f:
#                 current_patient_data = f.read()
#             current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
#             if filename.split('/')[-1].split('.')[0]  not in examples.keys():
#                 current_feature_data = filename.split('/')[-1].split('.')[0]
#                 with open(filename, 'rb') as fr:
#                     feature = pickle.load(fr)
#                 examples[current_feature_data] = {
#                     "id" : current_patient_id,
#                     "txt_fn" : current_txt_file,   #txt,
#                     "feature" : feature,  #wav2vev2
#                     "num_location": get_num_locations(current_patient_data),
#                     "frequency" : get_frequency(current_patient_data),
#                     "location" : current_loc,
#                     "age" : get_age(current_patient_data),
#                     "sex" : get_sex(current_patient_data),
#                     "height" : get_height(current_patient_data),
#                     "weight" : get_weight(current_patient_data),
#                     "pregnancy_status" : get_pregnancy_status(current_patient_data),
#                     "murmur" : get_murmur(current_patient_data),
#                     "outcome" : get_outcome(current_patient_data)

#                 }
#             if get_murmur(current_patient_data) == self.murmur_classes[0]:
#                 murmur_lable[i] = 1
#             elif get_murmur(current_patient_data) == self.murmur_classes[1]:
#                 murmur_lable[i] = 0
#             else :
#                 murmur_lable[i] = 0
                
#             if get_outcome(current_patient_data) == self.outcome_classes[0]:
#                 outcome_lable[i] = 1
#             elif get_outcome(current_patient_data) == self.outcome_classes[1]:
#                 outcome_lable[i] = 0
                
            
#         self.examples = examples
#         self.examples_list = list(examples.keys())
        self.feature_lst = feature_lst
        self.murmur_lable = murmur_lable
        self.outcome_lable = outcome_lable
        
        
        
        
        
#         if len(self.examples_list) != self.num_feature_files:
#             raise ValueError('length is different!')
    def __len__(self):
        return len(self.feature_lst)
    
        
    def __getitem__(self, index):
        
        # patient info
#         current_feature_dict = self.examples[self.examples_list[index]]
#         current_patient_data = load_patient_data(current_feature_dict["txt_fn"])

        
        
        feature = self.feature_lst[index]
        current_murmur = self.murmur_lable[index]
        current_outcome = self.outcome_lable[index]
        
        
#         #log mel feature
#         log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
#                                                          sr= 4000,
#                                                          n_mels=128,
#                                                          n_fft=400, 
#                                                          hop_length=128, 
#                                                          win_length=400))
        
#         # age
#         current_patient_age = current_wav_dict["age"]
#         current_age_group = np.zeros(6, dtype=np.float32)
#         if current_patient_age in self.age_classes:
#             j = self.age_classes.index(current_patient_age)
#             current_age_group[j] = 1.0
#         else :
#             current_age_group[5] = 1.0
            
            
#         # sex
#         sex = current_wav_dict["sex"]
#         sex_feature = np.zeros(2, dtype=np.float32)
#         if compare_strings(sex, 'Femalraw_datae'):
#             sex_feature[0] = 1.0
#         elif compare_strings(sex, 'Male'):
#             sex_feature[1] = 1.0
            
#         # height and weight.
#         height = current_wav_dict["height"]
#         weight = current_wav_dict["weight"]
        
#         ## simple impute
#         if math.isnan(height) :
#             height = 110.846  #mean
#         if math.isnan(weight) :
#             weight = 23.767   #mean
            
#         height_weight = np.array([height, weight], dtype=np.float32)
        
        
#         # Extract pregnancy
#         preg_feature = np.zeros(2, dtype=np.float32)
#         is_pregnant = current_wav_dict["pregnancy_status"]
#         if is_pregnant == True:
#             preg_feature[0] = 1.0
#         elif is_pregnant == False:
#             preg_feature[1] = 1.0

#         # Extract location
#         location = current_wav_dict["location"]
#         num_recording_locations = len(self.recording_locations)
#         loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
#         if location in self.recording_locations:
#             j = self.recording_locations.index(location)
#             loc_feature[j] = 1.0


        

        # label
        
#         current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
#         murmur = get_murmur(current_patient_data)
#         if murmur in self.murmur_classes:
#             j = self.murmur_classes.index(murmur)
#             current_murmur[j] = 1
#         else:
#             current_murmur[1] = 1   #unknown absent에 합침


#         current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
#         outcome = get_outcome(current_patient_data)
#         if outcome in self.outcome_classes:
#             j = self.outcome_classes.index(outcome)
#             current_outcome[j] = 1
            


        
        return feature, current_murmur, current_outcome

In [18]:
# Find data files.
# if verbose >= 1:
#     print('Finding data files...')

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())


GPU_NUM = "0" # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

# Find the patient data files.
patient_files = find_patient_files(args.train_data_folder)
num_patient_files = len(patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')
0 == 0.0
# Create a folder for the model if it does not already exist.
os.makedirs(args.model_path, exist_ok=True)
os.makedirs(args.logdir, exist_ok=True)

open(args.logdir + 'score.txt', "a+")
score_file = open(args.logdir + 'score.txt', "a+")


model_folder = args.model_path
log_dir = args.logdir
logger = get_logger(log_dir + '/' + model_folder)

# Define the model
wav2vec2_lcnn_model = Wav2Vec2_LCNN("murmur")
wav2vec2_lcnn_model = wav2vec2_lcnn_model.cuda(device)

# Define the metrics
# accuracy = Accuracy(num_classes=2)
# f1 = F1(num_classes=2,average='micro')

# optimizer, loss
opt = torch.optim.Adam(wav2vec2_lcnn_model.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
criterion = torch.nn.BCELoss()


#data loader
trainset = train_loader(args.train_data_folder )
validset = train_loader(args.valid_data_folder )
# point_sampler = RandomSampler(trainset)

dataloader_train = DataLoader(dataset=trainset,
                    batch_size=args.minibatchsize_train,
                           shuffle =True,
                           num_workers=args.train_num_workers)
dataloader_valid = DataLoader(dataset=validset,
                    batch_size=args.minibatchsize_valid,
                           num_workers=args.dev_num_workers)

Available devices  1
08-14 01:16 [MainProcess, 19490] [DEBUG]  Starting new HTTPS connection (1): huggingface.co:443
08-14 01:16 [MainProcess, 19490] [DEBUG]  Starting new HTTPS connection (1): huggingface.co:443
08-14 01:16 [MainProcess, 19490] [DEBUG]  https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/config.json HTTP/1.1" 200 0
08-14 01:16 [MainProcess, 19490] [DEBUG]  https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/config.json HTTP/1.1" 200 0
08-14 01:16 [MainProcess, 19490] [DEBUG]  Starting new HTTPS connection (1): huggingface.co:443
08-14 01:16 [MainProcess, 19490] [DEBUG]  Starting new HTTPS connection (1): huggingface.co:443
08-14 01:16 [MainProcess, 19490] [DEBUG]  https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
08-14 01:16 [MainProcess, 19490] [DEBUG]  https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/pytorch_model.bin HTTP/1.1" 302 0


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [117]:
trainset[0][0].shape

(120000,)

In [19]:
a,b, c = next(iter(dataloader_train))

In [21]:
a.shape

torch.Size([6, 80000])

In [25]:
torch.cuda.empty_cache()

In [11]:
torch.cuda.empty_cache()

murmur_classes = ['Present','Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']
if args.mode=='murmur':
    classes =  murmur_classes 
elif args.mode =='outcome':
    classes = outcome_classes


    
all_file_train = len(trainset)
dataloader_train_len = len(dataloader_train)




for iter_ in range(args.end_iter):
    train_pred_label,  train_gt_label = [], []
    start_time = time.time()
    running_loss = 0.0
    idx = 0
    wav2vec2_lcnn_model.train()
    for feature, current_murmur, _ in tqdm(dataloader_train): ###current_outcome
        feature = feature.cuda()
        feature = torch.tensor(feature, dtype=torch.float32)
#         current_age_group = current_age_group.cuda()
#         sex_feature = sex_feature.cuda()
#         height_weight = height_weight.cuda()
#         preg_feature = preg_feature.cuda()
#         loc_feature = loc_feature.cuda()
        current_murmur = current_murmur.cuda()
#         current_outcome = current_outcome.cuda()






        output_train = wav2vec2_lcnn_model(feature)   # output
        output_train = output_train.squeeze(1)


        output_preds_labels = (torch.ceil(output_train-0.4)).data.cpu().numpy()
        train_pred_label.extend(list(output_preds_labels))
        train_gt_label.extend(list(current_murmur.data.cpu().numpy()))
        
#         murmur_weight = [4 if i==1 else 1 for i in list(current_murmur.data.cpu().numpy())]
        
#         criterion = torch.nn.BCELoss(weight = torch.tensor(murmur_weight).cuda())
        loss = criterion(output_train.to(torch.float32), current_murmur.to(torch.float32))
                        
        


        idx +=1              

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item()

    train_f1 = f1_score(y_true= train_gt_label, y_pred= train_pred_label)
    
    
    scheduler.step()
    print("lr: ", opt.param_groups[0]['lr'])
    logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ".format(iter_, 
                 running_loss/dataloader_train_len,train_f1))

    # eval
    all_file_valid = len(validset)
    dataloader_valid_len = len(dataloader_valid)
    wav2vec2_lcnn_model.eval()

    with torch.no_grad():
        running_loss_dev = 0.0
        index = 0
        dev_pred_label,  dev_gt_label = [], []
        for feature,  current_murmur, _ in tqdm(dataloader_valid):  #current_outcome
            feature_dev = feature.cuda()
            feature_dev = torch.tensor(feature_dev, dtype=torch.float32)
#             current_age_group_dev = current_age_group.cuda()
#             sex_feature_dev = sex_feature.cuda()
#             height_weight_dev = height_weight.cuda()
#             preg_feature_dev = preg_feature.cuda()
#             loc_feature_dev = loc_feature.cuda()
            current_murmur_dev = current_murmur.cuda()
#             current_outcome_dev = current_outcome.cuda()


            output_dev = wav2vec2_lcnn_model(feature_dev)
            output_dev = output_dev.squeeze(1)

            
            
            output_preds_labels_dev = (torch.ceil(output_dev-0.4)).data.cpu().numpy()
            dev_pred_label.extend(list(output_preds_labels_dev))
            dev_gt_label.extend(list(current_murmur_dev.data.cpu().numpy()))
            
            
#             murmur_weight_dev = [4 if i==1 else 1 for i in list(current_murmur_dev.data.cpu().numpy())]
        
#             criterion = torch.nn.BCELoss(weight = torch.tensor(murmur_weight_dev).cuda())
            loss_dev = criterion(output_dev.to(torch.float32), current_murmur_dev.to(torch.float32))
            
            
            running_loss_dev += loss_dev.item()
                        

            index = index+1   
            
        dev_preds_onehot_label = np.array([[1.0,0.0,0.0] if i == 1 else [0.0,0.0,1.0] for i in dev_pred_label])    
        dev_gt_onehot_label = np.array([[1.0,0.0,0.0] if i == 1 else [0.0,0.0,1.0] for i in dev_gt_label])
            
        

        dev_f1 = f1_score(y_true= dev_gt_label, y_pred= dev_pred_label)
        weighted_acc = compute_weighted_accuracy(dev_gt_onehot_label, dev_preds_onehot_label, classes)

        



    logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f,Valid F1=%.4f, Valid weighted acc=%.4f" %( iter_,
            running_loss/dataloader_train_len, running_loss_dev/dataloader_valid_len, dev_f1,weighted_acc ))
    score_file.write("%d epoch, VALID LOSS %f, VALID F1 %2.2f%%, VALID WEIGHTED ACC %2.2f%%\n"%(iter_, 
                            running_loss_dev/dataloader_valid_len, dev_f1, weighted_acc ))
    score_file.flush()

    torch.save(wav2vec2_lcnn_model.state_dict(), os.path.join(model_folder, "{}_{}.model".format(args.model_name, iter_)))

    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)


# if verbose >= 1:
#     print('Done.')

100%|█████████████████████████████████████████| 422/422 [01:59<00:00,  3.54it/s]

lr:  9.1e-07
08-13 16:32 [MainProcess, 19490] [INFO ]  Iteration:0, train loss = 0.724867 ,train F1 = 0.320398 



100%|█████████████████████████████████████████| 158/158 [00:06<00:00, 23.72it/s]


08-13 16:32 [MainProcess, 19490] [INFO ]  valid Epoch:0, Train loss=0.7249, Valid loss=0.6923,Valid F1=0.3482, Valid weighted acc=0.5718
08-13 16:32 [MainProcess, 19490] [INFO ]  Time used for each epoch training: 126.40052390098572 seconds.
08-13 16:32 [MainProcess, 19490] [INFO ]  **************************************************


100%|█████████████████████████████████████████| 422/422 [02:01<00:00,  3.48it/s]

...

100%|█████████████████████████████████████████| 422/422 [02:01<00:00,  3.46it/s]

lr:  5.288242786788326e-10
08-13 19:23 [MainProcess, 19490] [INFO ]  Iteration:79, train loss = 0.454399 ,train F1 = 0.744615 



100%|█████████████████████████████████████████| 158/158 [00:06<00:00, 23.69it/s]

08-13 19:23 [MainProcess, 19490] [INFO ]  valid Epoch:79, Train loss=0.4544, Valid loss=0.4302,Valid F1=0.7280, Valid weighted acc=0.7971


08-13 19:23 [MainProcess, 19490] [INFO ]  Time used for each epoch training: 129.06274843215942 seconds.
08-13 19:23 [MainProcess, 19490] [INFO ]  **************************************************


In [35]:
train_f1 = f1_score(y_true= train_gt_label[:100], y_pred= train_pred_label[:100])

In [36]:
train_f1

0.1875

In [37]:
len(train_gt_label)

2532

In [25]:
train_pred_label[:10]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [27]:
output_dev.shape

torch.Size([3])

In [28]:
output_preds_labels_dev

array([-0., -0., -0.], dtype=float32)

In [31]:
dev_gt_label.count(1)

133

In [32]:
dev_pred_label.count(1)

0

In [34]:
len(dev_gt_label)

631

In [12]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load('/home/jh20/narin/physionet/hmd_LCNN' + '/' + 'hmd_LCNN_model_murmur_19.model')
    outcome_train_model = torch.load('/home/jh20/narin/physionet/hmd_CNN_outcome' + '/' + 'hmd_CNN_outcome_model_25.model')
    return [murmur_train_model, outcome_train_model]

In [14]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022/model")

from CNN_outcome import CNN as CNN_outcome

In [ ]:
import librosa
import math
import random

def get_features(data, recordings):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    
    
    feature_dict={}
    
    feature_dict['mel']=[]
    for i in range(num_recordings):
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = recordings[i].astype(np.float32),
                                                         sr= 4000,
                                                             n_mels=128,
                                                             n_fft=400, 
                                                             hop_length=128, 
                                                             win_length=400))
        feature_dict['mel'].append(log_mel_feature)
    
    recording_info_lst = data.split('\n')[1:num_recordings+1]
    feature_dict['wav2vec2'] = []
    for i in range(num_recordings):
        pk_feature_fnm = recording_info_lst[i].split(' ')[2].replace('wav','pickle')
        with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/'+ pk_feature_fnm ,'rb') as fr:
            feature = pickle.load(fr)
        feature_dict['wav2vec2'].append(feature)
    

    # age
    current_patient_age = get_age(data)
    current_age_group = np.zeros(6, dtype=np.float32)
    if current_patient_age in age_classes:
        j = age_classes.index(current_patient_age)
        current_age_group[j] = 1.0
    else :
        current_age_group[5] = 1.0

    feature_dict['age']=current_age_group



    # sex
    sex = get_sex(data)
    sex_feature = np.zeros(2, dtype=np.float32)
    if compare_strings(sex, 'Female'):
        sex_feature[0] = 1.0
    elif compare_strings(sex, 'Male'):
        sex_feature[1] = 1.0

    feature_dict['sex']=sex_feature

    # height and weight.
    height = get_height(data)
    weight = get_weight(data)

    ## simple impute
    if math.isnan(height) :
        height = 110.846  #mean
    if math.isnan(weight) :
        weight = 23.767   #mean

    height_weight = np.array([height, weight], dtype=np.float32)


    feature_dict['hw']=height_weight

    # Extract pregnancy
    preg_feature = np.zeros(2, dtype=np.float32)
    is_pregnant = get_pregnancy_status(data)
    if is_pregnant == True:
        preg_feature[0] = 1.0
    elif is_pregnant == False:
        preg_feature[1] = 1.0

    feature_dict['preg']=preg_feature

    # Extract location
    feature_dict['loc']=[]
    locations = get_locations(data)
    for j in range(num_recordings):
        
        num_recording_locations = len(recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if locations[j] in recording_locations:
            idx = recording_locations.index(locations[j])
            loc_feature[idx] = 1.0
            
        feature_dict['loc'].append(loc_feature)




    # label

    current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)
    murmur = get_murmur(data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    
    feature_dict['murmur']=current_murmur
    
    current_outcome = np.zeros(num_outcome_classes, dtype=np.float32)
    outcome = get_outcome(data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1

    feature_dict['outcome']=current_outcome
    
    


    return feature_dict

In [ ]:
def run_challenge_model(model, data, recordings, verbose):
    murmur_model , outcome_model = model
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print ('Available devices ', torch.cuda.device_count())
        
    GPU_NUM = args.gpu # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU

    
    # Define the model
    murmur_classifier = LCNN_murmur("murmur")
    murmur_classifier = murmur_classifier.cuda(device)
    
    outcome_classifier = CNN_outcome()
    outcome_classifier = outcome_classifier.cuda(device)
    
    murmur_classifier.load_state_dict(murmur_model)
    outcome_classifier.load_state_dict(outcome_model)
    
#     wav_files = glob.glob(data_folder + '*.wav')
#     num_wav_files = len(wav_files)
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    recordings_list=[]
    for i in range(num_recordings):
        recording = recordings[i]
        length = 20*4000  
        if recording.shape[0] <= length:
            shortage = length - recording.shape[0]
            recording = np.pad(recording, (0, shortage), 'wrap')
        start_frame = np.int64(random.random()*(recording.shape[0]-length))
        recording = recording[start_frame:start_frame + length] 
        
        recordings_list.append(recording)
        

    features = get_features(data, recordings_list)
    
    predict_murmur_arr= np.zeros((num_recordings,3))
    predict_outcome_arr= np.zeros((num_recordings,2))
    
    for i in range(num_recordings):
        
        logmel = torch.tensor(features['mel'][i]).cuda()
        age = torch.tensor(features['age']).cuda()
        sex = torch.tensor(features['sex']).cuda()
        hw = torch.tensor(features['hw']).cuda()
        preg = torch.tensor(features['preg']).cuda()
        loc = torch.tensor(features['loc'][i]).cuda()
        wav2vec2_feature = torch.tensor(features['wav2vec2'][i]).cuda()
        
        logmel = logmel.unsqueeze(0)
        age = age.unsqueeze(0)
        sex = sex.unsqueeze(0)
        hw = hw.unsqueeze(0)
        preg = preg.unsqueeze(0)
        loc = loc.unsqueeze(0)
        wav2vec2_feature = wav2vec2_feature.unsqueeze(1)
        
        predict_murmur = F.log_softmax(murmur_classifier(wav2vec2_feature), dim=1)
        predict_outcome = F.log_softmax(outcome_classifier(logmel,age,sex,hw,preg,loc), dim=1)
        
        predict_murmur_arr[i,:]= predict_murmur.data.detach().cpu().numpy()
        predict_outcome_arr[i,:] = predict_outcome.data.detach().cpu().numpy()

    
    
    
   
    # Get classifier probabilities.
    idx1 = predict_murmur_arr.argmax(axis=0)[0]
    murmur_probabilities = predict_murmur_arr[idx1,] 
    idx2 = predict_outcome_arr.argmax(axis=0)[0]
    outcome_probabilities = predict_outcome_arr[idx2,]


    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [ ]:
import numpy as np, os, sys
# from helper_code import *
# from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')